#UE03 - SPARQL Query

Please complete the 10 tasks in the `2. SPARQL` sheet of `SemAI.jar` first, and then transfer the task descriptions and your solutiosn in executable form to this notebook.

## Preparation

Reuse imports and functions from https://github.com/jku-win-dke/SemAI/blob/main/V02_SPARQL.ipynb and load the solar system graph. Query the solar system graph to check that everything works fine. 

In [1]:
# Install required packages
!pip install -q rdflib

# Imports
import pandas as pd
from rdflib import Graph, Literal, RDF, URIRef, BNode, Namespace
from rdflib.namespace import FOAF , XSD , RDFS, NamespaceManager 

# Convenient Functions
def sparql_select(graph,query,use_prefixes=True):
  results = graph.query(query)          # execute the query against the graph, resulting in a rdflib.plugins.sparql.processor.SPARQLResult
  rows = []                             # a list of dictionaries, as intermediate format to construct the pandas DataFrame
  for result in results:                # iterate over the result set of the query, a result is an instance of rdflib.query.ResultRow
    row = {}                            #     create a dictionary to hold a single row of the result
    for var in results.vars:            #     iterate over the variables of the SPARQLResult to add a dictionary entry for each variable
      if (isinstance(result[var],URIRef) and use_prefixes):
        row[var] = result[var].n3(graph.namespace_manager)   # use namespace prefixes to shorten URIs
      else:
        row[var] = result[var]                  
    rows.append(row)                    #     add the dictionary (row) to the list 
  return pd.DataFrame(rows,columns=results.vars)        
                                        # return a pandas DataFrame constructed from the list of dictionaries, with the variables from the result set as columns      

def sparql_construct(graph, query):
  result_graph = Graph(namespace_manager = g.namespace_manager)  # create a Graph object that reuses the namespace prefixes of the original graph
  result_graph += graph.query(query)                             # execute the construct query against the original graph and add the resulting graph to the new one
  return result_graph

def sparql_ask(graph, query):
  return bool(graph.query(query))      # an ASK query has a boolean result, which should be returned as such

# Load solar system graph
g = Graph()
g.parse("https://raw.githubusercontent.com/jku-win-dke/SemAI/main/data/solarsystem.ttl",format="turtle")

# Query solar system graph (to check that everything works fine)
df = sparql_select(g,"""
  SELECT ?planet ?apoapsis ?apoapsis_uom
  WHERE { 
    ?planet rdf:type dbo:Planet . 
    OPTIONAL { ?planet v:apoapsis [rdf:value ?apoapsis ; v:uom ?apoapsis_uom ].  }
  }
""")
df

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 528.1/528.1 KB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 KB 2.9 MB/s eta 0:00:00


,planet,apoapsis,apoapsis_uom
0,:Mercury,0.467,unit:AU
1,:Venus,0.728,unit:AU
2,:Earth,1.017,unit:AU
3,:Earth,149597871,unit:KM
4,:Mars,1.666,unit:AU
5,:Jupiter,5.4588,unit:AU
6,:Saturn,9.0412,unit:AU
7,:Uranus,20.11,unit:AU
8,:Neptune,30.33,unit:AU



## Task 1 (1 pt)

Geben Sie alle Zwergplaneten (Instanzen der Klasse dbo:DwarfPlanet) aus und falls vorhanden deren genaue Entsprechung (skos:exactMatch). Ordnen Sie das Ergebnis aufsteigend nach den URIs der Zwergplaneten.

In [2]:
# TODO: include and execute the query, display the result
query = """
SELECT *
WHERE {
?d rdf:type dbo:DwarfPlanet .
OPTIONAL {
?d skos:exactMatch ?match.
}
}
ORDER BY ?d 
"""

df = sparql_select(g, query)
df

,match,d
0,dbr:1_Ceres,:Ceres
1,None,:Eris
2,None,:Haumea
3,None,:Makemake
4,dbr:Pluto,:Pluto


## Task 2 (1 pt)

Ermitteln Sie alle Sterne, ihr exactMatch, und ihre Masse (geben Sie den Wert und die Maßeinheit aus). Ordnen Sie das Ergebnis nach der URI der Sterne

In [3]:
# TODO: include and execute the query, display the result
query = """
SELECT *
WHERE { 
?star rdf:type dbo:Star .
?star skos:exactMatch ?match .
?star v:mass [ rdf:value ?massVal ].
?star v:mass [ v:uom ?massUoM].
}
ORDER BY ?star
"""

df = sparql_select(g, query)
df

,star,massUoM,massVal,match
0,:AlphaCentauriA,v:SolarMass,1.1,wd:Q2090157
1,:AlphaCentauriB,v:SolarMass,0.9,wd:Q1052548
2,:ProximaCentauri,v:SolarMass,0.1221,wd:Q14266
3,:Sun,unit:KG,1.9884e+30,dbr:Sun


## Task 3 (1 pt)

Die Planeten unseres Sonnensystems und ihre jeweilige Anzahl an Monden. Unterscheiden Sie die im RDF-Graph beschriebenen Monde und die im RDF-Graph erfasste Anzahl von Monden. Sortieren Sie nach den Planeten.

In [4]:
# TODO: include and execute the query, display the result
query = """
SELECT ?planet ?assertedNo (count(?noMoons) AS ?noOfDescribedMoons) 
WHERE 
{ 
 ?planet rdf:type dbo:Planet .
 ?planet v:nrOfMoons ?assertedNo .
 OPTIONAL {
  ?noMoons rdf:type dbo:Satellite;
  v:orbits ?planet
 } 
}
GROUP BY ?planet ?assertedNo
ORDER BY ?planet
"""

df = sparql_select(g, query)
df

,planet,assertedNo,noOfDescribedMoons
0,:Earth,1,1
1,:Jupiter,79,4
2,:Mars,2,2
3,:Mercury,0,0
4,:Neptune,14,0
5,:Saturn,82,2
6,:Uranus,27,0
7,:Venus,0,0


## Task 4 (1 pt)

Ermitteln sie für die Planeten in unserem Sonnensystem die durchschnittliche Anzahl an Monden (die auch im RDF-Graph beschrieben sind) pro Planet.

In [10]:
# TODO: include and execute the query, display the result

## Task 5 (1 pt)

Geben Sie die im RDF-Graph verwendeten Klassen und ihre Anzahl an Instanzen aus. Geben Sie nur Klassen mit mindestens 2 Instanzen aus. Ordnen Sie die Ausgabe nach der URI der Klassen.

In [12]:
# TODO: include and execute the query, display the result
query = """
SELECT (?o AS ?class) (count(?o) AS ?noOfInstances)
WHERE { ?s rdf:type ?o  }
GROUP BY ?o
HAVING (count(?o) >= 2)
ORDER BY ?class
"""

df = sparql_select(g, query)
df

,class,noOfInstances
0,dbo:DwarfPlanet,5
1,dbo:Planet,8
2,dbo:Satellite,9
3,dbo:Star,4


## Task 6 (1 pt)

*TODO: add task description*

In [ ]:
# TODO: include and execute the query, display the result

## Task 7 (1 pt)

Erzeugen sie mittels einer Construct-Query den angezeigten RDF-GRAPH (Expected Result). Reihenfolge ist unerheblich.

In [14]:
# TODO: include and execute the query, display the result
query = """
SELECT ?x ?y
WHERE { 
{?x rdf:type dbo:Star}
UNION 
{?x rdf:type dbo:Satellite;
v:orbits ?y}
}
ORDER BY ?x
"""

df = sparql_select(g, query)
df

,x,y
0,:AlphaCentauriA,None
1,:AlphaCentauriB,None
2,:Callisto,:Jupiter
3,:Deimos,:Mars
4,:Enceladus,:Saturn
5,:Europa,:Jupiter
6,:Ganymede,:Jupiter
7,:Io,:Jupiter
8,:Mimas,:Saturn
9,:Moon,:Earth


## Task 8 (1 pt)

*TODO: add task description*

In [ ]:
# TODO: include and execute the query, display the result

## Task 9 (1 pt)

Gibt es einen Stern im RDF-Graph, der massereicher als die Sonne ist.

In [22]:
# TODO: include and execute the query, display the result
question = """ASK {
SELECT (?starmass * 1.988E+30) ?mass
WHERE { 
{
SELECT ?starmass
WHERE { 
?star rdf:type dbo:Star;
 v:mass [ rdf:value ?starmass; v:uom v:SolarMass];
}
}
OPTIONAL {
?s skos:exactMatch dbr:Sun;
v:mass [rdf:value ?mass] .
}
FILTER ((?starmass * 1.988E+30) > ?mass)
}
}
"""
print(sparql_ask(g, question))
# if(sparql_ask(g, question)):
#   print("Yes, some is bigger")
# else: 
#   print("No, nothing is bigger")

ParseException: ignored

## Task 10 (1 pt)

*TODO: add task description*

In [15]:
# TODO: include and execute the query, display the result
g2 = sparql_construct(g,"""
  DESCRIBE ?s
WHERE { 
?s v:radius [ rdf:value ?r] 
FILTER(?r> 20000)
FILTER(?r< 30000)
}
""")

print(g2.serialize(format='turtle'))


@prefix : <http://dke.jku.at/example/solarsystem/> .
@prefix dbo: <http://dbpedia.org/ontology/> .
@prefix dbr: <http://dbpedia.org/resource/> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix skos: <http://www.w3.org/2004/02/skos/core#> .
@prefix unit: <http://qudt.org/vocab/unit/> .
@prefix v: <http://dke.jku.at/example/examplevocabulary/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

:Neptune a dbo:Planet ;
    v:apoapsis [ v:uom unit:AU ;
            rdf:value 30.33 ] ;
    v:nrOfMoons 14 ;
    v:orbitalPeriod [ v:uom unit:YR ;
            rdf:value 164.8 ] ;
    v:orbits :Sun ;
    v:radius [ v:uom unit:KM ;
            rdf:value 24622 ] ;
    v:temperature [ v:uom unit:Deg_C ;
            rdf:value -201 ] ;
    skos:exactMatch dbr:Neptune .

:Uranus a dbo:Planet ;
    v:apoapsis [ v:uom unit:AU ;
            rdf:value 20.11 ] ;
    v:nrOfMoons 27 ;
    v:orbitalPeriod [ v:uom unit:YR ;
            rdf:value 84.0205 ] ;
    v:orbits :Sun ;
    v:radius